# Script for filling in session numbers and within-session test numbers

This script 'guesses' what session numbers and within-session test numbers should be, then adds them to a copy of a test results file. You can then manually copy the values from this file into your main test results file.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_test_results = pd.read_csv('Files/test_results.csv')
df_test_results

,Test_Number,Session_Number,Within_Session_Test_Number,Unix_Test_Start_Time,Local_Test_Start_Time,Unix_Test_End_Time,Local_Test_End_Time,Verse_ID,Verse_Code,Verse,...,Test_Seconds,Error_Rate,Error_and_Backspace_Rate,Marathon_Mode,Player,Mode,Tag_1,Tag_2,Tag_3,Notes
0,1,NaN,NaN,1750909743,2025-06-25T23:49:03-0400,1750909747,2025-06-25T23:49:07-0400,1,Genesis_1:1,"In the beginning, God created heaven and earth.",...,4.514593,0.021277,0.042553,0,KJB3,SP,Cherry_Red,NaN,NaN,NaN
1,2,NaN,NaN,1750910427,2025-06-26T00:00:27-0400,1750910430,2025-06-26T00:00:30-0400,1,Genesis_1:1,"In the beginning, God created heaven and earth.",...,3.115154,0.000000,0.000000,0,KJB3,SP,Cherry_Red,NaN,NaN,NaN
2,3,NaN,NaN,1750910839,2025-06-26T00:07:19-0400,1750910854,2025-06-26T00:07:34-0400,2,Genesis_1:2,"But the earth was empty and unoccupied, and da...",...,14.912085,0.120567,0.205674,1,KJB3,SP,Cherry_Red,NaN,NaN,NaN
3,4,NaN,NaN,1750910854,2025-06-26T00:07:34-0400,1750910860,2025-06-26T00:07:40-0400,3,Genesis_1:3,"And God said, ""Let there be light."" And light ...",...,5.741471,0.018868,0.037736,1,KJB3,SP,Cherry_Red,NaN,NaN,NaN
4,5,NaN,NaN,1750910860,2025-06-26T00:07:40-0400,1750910869,2025-06-26T00:07:49-0400,4,Genesis_1:4,"And God saw the light, that it was good; and s...",...,9.170210,0.078652,0.157303,1,KJB3,SP,Cherry_Red,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,1327,51.0,36.0,1754966864,2025-08-11T22:47:44-0400,1754966871,2025-08-11T22:47:51-0400,14161,Job_22:28,"You will decide on something, and it will come...",...,7.273231,0.053763,0.064516,0,KJB3,SP,Cherry_Red,Home,NaN,NaN
1327,1328,51.0,37.0,1754966875,2025-08-11T22:47:55-0400,1754966881,2025-08-11T22:48:01-0400,14161,Job_22:28,"You will decide on something, and it will come...",...,6.170374,0.000000,0.000000,0,KJB3,SP,Cherry_Red,Home,NaN,NaN
1328,1329,51.0,38.0,1754966915,2025-08-11T22:48:35-0400,1754966919,2025-08-11T22:48:39-0400,13,Genesis_1:13,"And it became evening and the morning, the thi...",...,4.040784,0.056604,0.075472,0,KJB3,SP,Cherry_Red,Home,NaN,NaN
1329,1330,51.0,39.0,1754966921,2025-08-11T22:48:41-0400,1754966924,2025-08-11T22:48:44-0400,13,Genesis_1:13,"And it became evening and the morning, the thi...",...,3.114044,0.000000,0.000000,0,KJB3,SP,Cherry_Red,Home,NaN,NaN


In [3]:
session_num_col = df_test_results.columns.get_loc('Session_Number')
within_session_test_num_col = df_test_results.columns.get_loc(
'Within_Session_Test_Number')
session_num_col, within_session_test_num_col

(1, 2)

Starting our Session_Number count at 1:

In [4]:
df_test_results.iloc[0, session_num_col] = 1
df_test_results.iloc[0, within_session_test_num_col] = 1

In [5]:
session_num = 1 # This number will get updated as needed within
# the following for loop.
within_session_test_num = 1 # Same for this number
for i in range(1, len(df_test_results)):
    duration_since_last_test = df_test_results.iloc[i][
    'Unix_Test_Start_Time'] - df_test_results.iloc[i-1][
'Unix_Test_End_Time'] 
        # If it has been 10 or more minutes (600 seconds) since
        # the end of the last test, we'll count this as the 
        # start of a new session. (This isn't how session numbers
        # are actually incremented within TTTB, but this approach
        # will work for now.)
    if duration_since_last_test >= 600:
        session_num += 1 # A new session has begun
        within_session_test_num = 1 # Resetting within-session test number
    else:
        within_session_test_num += 1 # We'll assume that this session
        # is the same as the previous one; thus, we'll increment the
        # within-session test number while keeping the session number
        # the same.
    # print(duration_since_last_test, session_num)
    df_test_results.iloc[i, session_num_col] = session_num
    df_test_results.iloc[
        i, within_session_test_num_col] = within_session_test_num

In [6]:
df_test_results.to_csv('updated_results.csv', index = False)